The goal is to write a simple object detection pipeline invoking YOLOX.

In [1]:
import numpy as np
import torch
import torchvision
import cv2

There are many YOLOX versions based on size, depth and efficiency and we select which yolox version to use by using the `get_exp` function provided by YOLOX.

In [6]:
import sys
sys.path.insert(0, "./YOLOX/")
from YOLOX.yolox.exp.build import get_exp_by_name

In [10]:
YOLOX_VERSION = "yolox-s" #alternatives: yolox-m, yolox-l, yolox-x, yolox-tiny, yolox-nano, yolov3
yolo_exp = get_exp_by_name(YOLOX_VERSION)

In [5]:
mask = np.array([True, False])
b =  ~mask
b

array([False,  True])

In [36]:
class A:
    def __init__(self, x):
        self.x = x
        
class B(A):
    def __init__(self, y):
        super().__init__(y+1)
        self.y = y
        
    def __repr__(self):
        return f" self.y = {self.y}, self.x = {self.x}"

In [38]:
b = B(4)
b

 self.y = 4, self.x = 5

In [42]:
a = np.arange(6).reshape((2,3))
a[[0,1], [2,1]]

array([[ 0,  1, -1],
       [ 3, -1,  5]])

In [44]:
~np.ones(shape=3, dtype=bool)

array([False, False, False])

In [45]:
a = np.array([[1,2],[3,4],[5,6]])
x,y = a.T
x,y

(array([1, 3, 5]), array([2, 4, 6]))